`참고: 덕성여대 Nano-degree 교과 <언어 빅데이터 이해와 활용>,<언어 빅데이터>`

```
작성자: 박민준
참고문헌:
Bird, S., Klein, E., & Loper, E. (2009). Natural language processing with Python: analyzing text with the natural language toolkit. " O'Reilly Media, Inc.".
이기창. 한국어 임베딩. 에이콘.
```
https://www.nltk.org/book/

# NLTK ngram 모듈 다운로드

In [ ]:
!gdown https://drive.google.com/uc?id=1v38ZSptVzQWP8FK5iJk-ojU3Z1xHFR1g

Downloading...
From (original): https://drive.google.com/uc?id=1v38ZSptVzQWP8FK5iJk-ojU3Z1xHFR1g
From (redirected): https://drive.google.com/uc?id=1v38ZSptVzQWP8FK5iJk-ojU3Z1xHFR1g&confirm=t&uuid=cd348c1b-b9ae-4fbc-92ce-bdba32c6311a
To: /content/ngram.py
100% 9.97k/9.97k [00:00<00:00, 19.0MB/s]


In [ ]:
from ngram import NgramModel

In [ ]:
corpus = 'cat meows and dog barks. a cat sat on the mat . the dog barked at the cat . bowwow'.split()

In [ ]:
corpus

['cat',
 'meows',
 'and',
 'dog',
 'barks.',
 'a',
 'cat',
 'sat',
 'on',
 'the',
 'mat',
 '.',
 'the',
 'dog',
 'barked',
 'at',
 'the',
 'cat',
 '.',
 'bowwow']

In [ ]:
len(corpus)

20

# 2. Ngram 확률 분포 작성

In [38]:
from nltk.probability import MLEProbDist, LaplaceProbDist, LidstoneProbDist

## 최대우도추정(MLE)로 확률 분포 구하기

$$
P(w_i) = \frac{C(w_i)}{N}
$$

- $P(w_i)$: 단어 $w_i$의 확률
- $C(w_i)$: 단어 $w_i$의 빈도수
- $N$: 전체 코퍼스에서의 총 단어(tokens) 수

'cat meows and dog barks. a cat sat on the mat . the dog barked at the cat . bowwow'

In [ ]:
from nltk.probability import FreqDist

In [ ]:
print(corpus)

['cat', 'meows', 'and', 'dog', 'barks.', 'a', 'cat', 'sat', 'on', 'the', 'mat', '.', 'the', 'dog', 'barked', 'at', 'the', 'cat', '.', 'bowwow']


In [ ]:
fd= FreqDist(corpus)

In [ ]:
fd

FreqDist({'cat': 3, 'the': 3, 'dog': 2, '.': 2, 'meows': 1, 'and': 1, 'barks.': 1, 'a': 1, 'sat': 1, 'on': 1, ...})

In [ ]:
a = MLEProbDist(fd) # 최대우도추정 진행

In [ ]:
a.prob('cat')     # 3/20

0.15

In [ ]:
m3 = NgramModel(3, corpus, estimator= lambda fdist, bins: MLEProbDist(fdist))     # 2로 하면 2-gram, 3으로 하면 3-gram

In [ ]:
' '.join(m3.generate(50))       # 50개를 생성 -> corpus 이후 . the dog~를 예측

'cat meows and dog barks. a cat sat on the mat . the dog barked at the cat . bowwow the cat . bowwow the cat . bowwow the mat . the dog barked at the cat . bowwow meows and dog barks. a cat sat on the mat .'

## 데이터 희소성 문제(Data Sparsity Problem) 대처하기

### Laplace smoothing (add-1 smoothing)

$$
P(w_i) = \frac{C(w_i) + 1}{N + |V|}
$$

분자에 특정 토큰 횟수에 1을 더해준다. -> 타입수도 분모에 더해줘 왜곡을 줄임

In [ ]:
m2 = NgramModel(3, corpus, estimator=lambda fdist, bins: LaplaceProbDist(fdist, bins=len(fdist)))   # bins: corpus의 type수

In [ ]:
m2._ngrams

{('.', 'the', 'dog'),
 ('START', 'START', 'cat'),
 ('START', 'cat', 'meows'),
 ('a', 'cat', 'sat'),
 ('and', 'dog', 'barks.'),
 ('at', 'the', 'cat'),
 ('barked', 'at', 'the'),
 ('barks.', 'a', 'cat'),
 ('cat', '.', 'bowwow'),
 ('cat', 'meows', 'and'),
 ('cat', 'sat', 'on'),
 ('dog', 'barked', 'at'),
 ('dog', 'barks.', 'a'),
 ('mat', '.', 'the'),
 ('meows', 'and', 'dog'),
 ('on', 'the', 'mat'),
 ('sat', 'on', 'the'),
 ('the', 'cat', '.'),
 ('the', 'dog', 'barked'),
 ('the', 'mat', '.')}

#### Generate text

In [ ]:
' '.join(m2.generate(30))

'cat meows and dog barks. a cat sat on the mat . the dog barked at the cat . bowwow at the cat . bowwow dog barks. a cat sat'

### Lidstone smoothing (add-γ smoothing)

$$
P(w_i) = \frac{C(w_i) + \gamma}{N + \gamma \cdot |V|}
$$

분모에 타입수*감마 , 분자에 감마를 더함

In [ ]:
m2 = NgramModel(2, corpus, estimator=lambda fdist, bins: LidstoneProbDist(fdist, 0.2))      # 감마 = 0.2

In [ ]:
m2._ngrams

{('.', 'bowwow'),
 ('.', 'the'),
 ('START', 'cat'),
 ('a', 'cat'),
 ('and', 'dog'),
 ('at', 'the'),
 ('barked', 'at'),
 ('barks.', 'a'),
 ('cat', '.'),
 ('cat', 'meows'),
 ('cat', 'sat'),
 ('dog', 'barked'),
 ('dog', 'barks.'),
 ('mat', '.'),
 ('meows', 'and'),
 ('on', 'the'),
 ('sat', 'on'),
 ('the', 'cat'),
 ('the', 'dog'),
 ('the', 'mat')}

#### Generate text

In [ ]:
' '.join(m2.generate(30))

'cat meows and dog barks. a cat . the cat . the dog barks. a cat sat on the mat . the mat . the cat sat on the cat'

# 3. (과제) 2-gram, 3-gram, 4-gram 확률 분포에 따른 문장 생성하기
* 더 큰 규모의 코퍼스 (예: 네이버 댓글 말뭉치 등)로 시도해 보세요.
* 각 n-gram 확률분포에 따른 문장 생성의 장점과 단점은 무엇일까요?

단점: 제대로 확률 예측을 하는데 위험도가 길어짐 -> 확률이 0이 나올 수 있음,
장점: 생성된 단어의 의미를 이해하는데 더 구체적, 직관적임

In [ ]:
from nltk.corpus.reader import TaggedCorpusReader

## Google Drive 와 연동하기 (corpus가 개인 구글 드라이브에 있을 경우)

In [2]:
# 내 구글 드라이브에 연동하기 (주의:이 컴퓨터의 다른 사용자가 접근할 수 있음)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 작업 디렉토리 설정 (개인마다 다르겠죠?)
root_path='/content/drive/MyDrive/Classroom/빅데이터시스템-01'

## Training corpus 준비
* Training corpus : 네이버 댓글 말뭉치

In [ ]:
reader = TaggedCorpusReader(root_path+"/data/tokenized/", 'ratings_mecab.txt')    # 단어(품사) 단위로 전처리된 말뭉치 텍스트 파일 이용

In [ ]:
len(reader.words())

3691415

In [ ]:
corpus = reader.words()

## 각각의 N-gram에 따른 문장 생성기 만들기
* n-gram 확률분포를 참조하여 조건부 확률의 연쇄에 따라, 주어진 문맥에서 가장 출현확률이 높은 다음 단어 출력하기
* `NgramModel`은 데이터 희소성 문제를 smoothing과 Katz Backoff 기법으로 해결하고 있습니다

In [13]:
# ngram 모듈 다운로드
!gdown https://drive.google.com/uc?id=1v38ZSptVzQWP8FK5iJk-ojU3Z1xHFR1g

Downloading...
From (original): https://drive.google.com/uc?id=1v38ZSptVzQWP8FK5iJk-ojU3Z1xHFR1g
From (redirected): https://drive.google.com/uc?id=1v38ZSptVzQWP8FK5iJk-ojU3Z1xHFR1g&confirm=t&uuid=6c196fb7-1cda-48ce-aaa9-16630830f656
To: /content/drive/MyDrive/Classroom/빅데이터시스템-01/data/ngram.py
100% 9.97k/9.97k [00:00<00:00, 16.3MB/s]


In [14]:
from ngram import NgramModel

In [ ]:
two_gram_model = NgramModel(2, corpus, estimator=lambda fdist, bins: LaplaceProbDist(fdist, bins=len(fdist)))

In [ ]:
three_gram_model = NgramModel(3, corpus, estimator=lambda fdist, bins: LaplaceProbDist(fdist, bins=len(fdist)))

In [ ]:
four_gram_model = NgramModel(4, corpus, estimator=lambda fdist, bins: LaplaceProbDist(fdist, bins=len(fdist)))

In [ ]:
# 2-gram 확률분포를 참조한 랜덤 문장 생성
' '.join(two_gram_model.generate(500))

"어릴 때 였 던 드라마 ~~~~♡ 영화 . 피디 잡 아 하 는 최고 의 주축 을 수 있 을 받 았 을까 . 귀여워 쏘우 풀 고 픈 이 라기에 도 함 이 란 이런 스토리 음악 까지 따로 따로 있 을까 ? ? SBS 의 짜증 나 니 ㅋㅋ 런닝맨 유재석 때문 에 웃음 한마디 로 온 남자 . ' 클레멘타인 이후 로 맞 아 ~~~♡ 기발 한 똥폼 1 0 년 후 4 빠 이 영화 중 갑 ♡ 너무 아니 지만 좋 으나 좀 비 정사 씬 by 박중훈 의 꿈 같 은데 왜 그리도 많 으니깐 점수 를 위한 거 라 쥬 의 관점 을 의미 도 없 고 6 , 번 더 재밌 어요 . .. 슈퍼 노바 지구 의 임수정 씨 를 만날 수 있 게 낫 더라 좋 아 해요 공공 의 신박 하 고 재 밋 는 사람 을 도와 감동 쫌 . ... 기자 평론 가 얼마나 많 은 기분 은 영화 박선영 닮 아서 전체 적 으로 감히 느와르 의 영화 도 더듬 자면 절대 기준 으로 별반 차이 남 는 영화 땜 에 미용실 미용사 의 영호 ㅓ네요 ^^ 영화 라고 생각 함 최고 였 습니다 . 볼 시간 에 치유 . .. 영화배우 님 그냥 장풍 놀이 . . 스토리 전개 안 보 고 있 는 졸업 , 코미디 영화 라서 타잔 보다 심리 를 잘 어우러진 . . . .. ㅡㅡ보는내내 지루 해 . 모리 준이치 는 좋 았 는데 보 고 유라 는 콩트 가 아깝 다 후반부 에 요 동남아 경찰 의 꼬맹이 연기 개차반 스러운 영화 에 무자비 한 건지 고소영 출연 한 포클랜드 사건 의 연기 가 내겐 사연 재미있 게 시청 률 이 다 빨린 다 봐야 하 다 ! ! 덕분 ( 케이블 께 진심 올 만 날리 던가 . 박유천 과 잣대 가 되 었 습니다 . ㅋㅋㅋ 아 꼇기에 만날 태경 에 이거 만들 었 던 죽음 을 완전히 베낀 표절 냄새 가 설정 들 어요 조니뎁 . ... 선배 로 사랑 스럽 다는 건가 들 ㄳㄳ 소재 인 한국 영화 감히 비평 이 다 크리스찬 베일 에 비해 평점 남김 소재 잡 게 현실 에서 섹 셉 , , 오래도록 가슴 아픈 역사 상 을 보 기 도 있 움 , 어거스터

In [ ]:
# 3-gram 확률분포를 참조한 랜덤 문장 생성
' '.join(three_gram_model.generate(500))

'어릴 때 봤 는데 . .. 곽재용 영화 는 역시 뭐니뭐니 해도 이특 강소라 가 진리 만세 ! ! 그 연약 하 고 고은아 는 발음 과 틱 장애 연기 가 너무 많 고 , 전개 도 결말 과 라스트 씬 에 반했 다는 달콤 한 영화 임 ㅠㅠㅠㅠ 시간 아껴야 죠 ~ 레슬리 닐슨 은 주인공 인 \' 인간 중독 "" 도 굿 . . 뭐 맨날 심사 위원 심사 가 너무 힘들 다 . 성경 구절 은 왜 주 는지 . ..;;;; 현재 우뢰매 나 후 뢰 시 맨 팬 이 라서 1 0 점 이 그렇게 갔 을까 ? 안 무섭 네 . . 1 2 3 : 2 3 4 5 6 8 이나 되 라 얍 ! ㅋ 참 유치 하 나 스 보리 는 흥미진진 한 비극 이 라고 ! 멍청이 ! 멍청이 ! 멍청이 ! 멍청이 들 땜 에 4 점 으로 올린다 . 심리극 편 과 상관 없이 재밌 게 봤 데이 재미 잇 다 앗 흥 ~ 잼 있 어 손 까락 에 꼽히 는 영화 ! 오늘 방금 OCN 에서 틀 어 막 고 소름 대 는 데 방법 이 없 다 내생 에 5 점 만화 의 산 으로 . .. 솔직히 초딩 떄였나 중 딩 아닐까 싶 습니다 ㅜㅠ 가장 미국 을 까 는 건 다릅니다 . ) 프랑스 티비 에서 해 준 고마운 영화 새삼 느끼 게 되 는 건 또 . .. 음 좀 미지근 한 것 이 다 날 지경 . . 행동 의 표본 아주 좋 았 음 . 몬스터주식회사 가 훨씬 좋 은 드라마 일 뿐 별루 네요 ~~~ 근데 결말 은 후 "" 날 따라 하 고 , 고춧가루 뿌리 고 잠적 해 버린 영화판 에서 가끔 씩 터져 주 는 달콤 하 지만 폭격기 가 한 국판 환상 특급 ` 을 너무 많이 다뤄져 자칫 진부 할 수 있 다 . . ㅠㅠ 님 들 다시 봤 지만 실제로 는 절대로 인기 끌 엇 다고 여자 때리 는 영화 . 잘 봤 습니다 . 영국 수상 대처 가 정말 매력 적 이탈리아어 배우 고 도 또 다시 신 은 평범 한 시나리오 와 이딴 연출 로 재미 가 있 을 여자 를 만날 줄 몰랐 는데 . .. 연기력 들 쩌 심 ㅠㅠ 와 . ,.,.,. 아 진짜 스토리 정말 욕 하 지 말 고 로보

In [ ]:
# 4-gram 확률분포를 참조한 랜덤 문장 생성
' '.join(four_gram_model.generate(500))

'어릴 때 보 고 충격 받 음 아오이 유우 맨 앞 에 있 는 아멜리에 가 너무 사랑 스럽 구요 ㅠ 보 는 내내 피 가 끓 는다 ! ! 그래픽 은 중국 풍 ~ 정서 는 일본풍 ~ 전형 적 뻔한 스토리 에 재미 도 있 음 근데 상영관 이 적 은 이유 를 모르 겠 다 . ★ 5 감독 이 김민종 배우 생명 끊 으려고 만든 영화 같 다 난 다시 보 기 를 반복 하 게 한 뒤 쇼핑 하 면서 돌아온다는 그 영화 인가요 영화 도 코미디 도 드라마 도 뭣 도 아니 게 수 나 이야기 로 끝납니다 . 그동안 재밌 게 보 았 다 . .. 하찮 지만 누구 에게 나 순간 이 내인생의 영화 가 아닌가 싶 으다 . ...! 간만에 다시 봐도 재밌 는가 ? ㅎ 박신양 짱 평점 낮 아서 드 렿 어요 . . 어렵 다니 ? 처음 부터 보 고 구매 할 것 을 굳이 영화 로 만들 어 버리 는 게 맞 을 듯 저질 무비 ~ 사라져 중국 인 들 의 처절 한 표정 연기 가 일품 이 에요 . .~! 참 재미있 었 지 ㅎ 샤론 누나 리즈 시절 소름 끼치 는 작품 이 될 영화 . . 시간 이 너무 . . 슬퍼요 . . 현실 적 이 라 어린 여아 들 은 무서워 . .. 진짜 . .. ㅋㅋ ㅋㅋ 뭐 야 퇴마 사 ? ㅋㅋ ㅋㅋ 엿 국 니 네 가 그렇 지 뭐 개 죽 은 거 잖아 징 편 인데 단편 스럽 다 . 밀폐 된 공간 에서 수준 높 은 호러 라고 평점 8 점 너무 높 은 관계 로 9 점 줌 느닷없이 찾아온 불행 이 사람 을 울려요 . ㅠ . . 아들 이 들 좋 아 하 는 사람 많 았 음 괜히 봤 다 . 연기 내공 은 정말 . .. 시청자 가 그 감정 을 그대로 느낄 수 있 던 충분 한 대본 이 었 다 . . 덱스터 ㅠ 영화 가 너무 무섭 다 . 한국 에 좋 은 영화 . 트 렌스 젠더 제이미 루너 를 추적 중 인데 여기 서 끊기 네 . .. 전 이거 매일 봐요 저 이거 팬 이 에요 사랑 하 는 사람 왜 적 어 놨 네 웅장 함 그 어떤 것 을 나타내 준 영화 . 최소한 의 재미 는 다 어디 로 . ..? 이야기 는 그

### 긍정적/부정적인 댓글 생성
* 3.3장 주석말뭉치 구축.ipynb 의 3.3 유형 말뭉치 리더(CategorizedTaggedCorpusReader)를 사용하여 Training Corpus를 category별로 학습시켜 봅시다

In [1]:
! pip install konlpy

# google colab에서 mecab을 사용하기 편하게 만들어주는 shell 파일도 실행시켜 줍니다.
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
# ! cd ./Mecab-ko-for-Google-Colab
# ! bash ./Mecab-ko-for-Google-Colab/install_mecab-ko_on_colab190912.sh
! bash ./Mecab-ko-for-Google-Colab/install_mecab-ko_on_colab_light_220429.sh

# shell 파일 출처: https://somjang.tistory.com/entry/Google-Colab에서-Mecab-koMecab-ko-dic-쉽게-사용하기 [솜씨좋은장씨]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 23.7 MB/s eta 0:00:00
Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 138 (delta 26), reused 22 (delta 8), pack-reused 91
Receiving objects: 100% (138/138), 1.72 MiB | 7.70 MiB/s, done.
Resolving deltas: 100% (65/65), done.
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2024-04-11 02:09:37--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 18.205.93.1, 18.205.93.2, 18.205.93.0, ...
Connecting to bitbucket.org (bitbucket.org)|18.205.93.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbus

In [30]:
%cd /content/drive/MyDrive/Classroom/빅데이터시스템-01/data

/content/drive/MyDrive/Classroom/빅데이터시스템-01/data


In [31]:
!mkdir corpus

In [32]:
input_file = 'processed_ratings_train.txt'

In [5]:
from konlpy.tag import Mecab

In [34]:
tokenizer = Mecab()

with open(input_file, encoding='utf-8') as fr:
  for i, line in enumerate(fr, start=1):
    try:
      comment, senti = line.split('␞') # 댓글, 감정레이블로 나눔
      tokenized = tokenizer.pos(comment.strip()) # 토크나이징, 단어 분리 뿐만 아니라 품사 분리까지 진행
      tagged_sent = ' '.join(list('/'.join((w,t)) for (w,t) in tokenized) )
      senti = int(senti)
      # 폴더/파일명 설정
      # 감정 레이블이 0이면 neg, 1이면 pos
      fname = f"corpus/doc{i:03d}_{'pos' if senti else 'neg'}.txt"
      with open(fname, 'w', encoding='utf-8') as ft:
        ft.write(tagged_sent)
        ft.write('\n')
      print(i)
    except Exception as e:
      print(e)
      continue
    if i == 1000: break # 전부 다 하면 15만개 정도... 오래 걸려요.

# print(tokenizer.pos(s2))
# print(' '.join(list('/'.join(wt) for wt in tokenizer.pos(s2)) ) )

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [8]:
from nltk.corpus.reader import CategorizedTaggedCorpusReader

In [35]:
reader = CategorizedTaggedCorpusReader('corpus', r'doc\d+_\w+\.txt', cat_pattern=r'doc\d+_(\w+)\.txt')

In [36]:
reader.categories()

['neg', 'pos']

In [39]:
pos_corpus = reader.words(categories='pos')
positive_model = NgramModel(3, pos_corpus, estimator=lambda fdist, bins: LaplaceProbDist(fdist, bins=len(fdist)))

In [40]:
neg_corpus = reader.words(categories='neg')
negative_model = NgramModel(3, neg_corpus, estimator=lambda fdist, bins: LaplaceProbDist(fdist, bins=len(fdist)))

In [41]:
' '.join(positive_model.generate(100))

'흠 . .. 사랑 . .. 포스터 보고 초딩 영화 줄 . ... 우리 모두 는 누군가 실제로 따라 했 으면 합니다 . . 또한 픽션 풍 정정 은 어 케 된 거 지 ;; 난 존나 재밌 던데 늑대 들 의 생각 은 19 세기 . 책 과 는 분명히 다른 시선 . 괜찮 은 영화 . .. 누이 랑 , 근친상간 이나 하 고 바라 는 것 · 크리스 터커 짱 이 죠 완전 잼 잇 게 봣습니닼ㅋ 옆 에 친구 중간 중간 노이 해 . ... 눈물 이 . . 영화 가 왜 이렇게 후덕 하 지'

In [42]:
' '.join(negative_model.generate(100))

'아 더 빙 . . 연예계 를 바라보 는 태도 를 고쳐야 한다 . 파괴 된 관객 들 . 너무 구식 으로 웃길려고 함 . . 연예계 를 바라보 는 태도 를 고쳐야 한다 . 파괴 된 관객 들 . 너무 구식 으로 웃길려고 함 . 보 긴 보 는데 진짜 아오 . . 토 나온다 . .. 도입부 를 제외 하 고 3 분 만 에 나왔 습니다 ( 웃음 ) 김혜선 은 @ 내일 이 오 면 @ 의 김순정 , 순정 이 역할 이 제일 이 다 . 2003 년 에 본 영화 중 최악 이 었'